#  Image Classification for Cattle CVD Detection via Retina Images

In [1]:
import sys
import pandas as pd
import numpy as np
import sklearn
import matplotlib
import keras


In [2]:
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

In [3]:
# import the heart disease dataset
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data"

# the names will be the names of each column in our pandas DataFrame
names = ['age',
        'sex',
        'cp',
        'trestbps',
        'chol',
        'fbs',
        'restecg',
        'thalach',
        'exang',
        'oldpeak',
        'slope',
        'ca',
        'thal',
        'class']

# read the csv
cleveland = pd.read_csv(url, names=names)

In [ ]:
# print the shape of the DataFrame, so we can see how many examples we have
print ('format(cleveland.shape')
print (cleveland.loc[1])

In [ ]:
# print the last twenty or so data points
cleveland.loc[280:]

In [ ]:
# remove missing data (indicated with a "?")
data = cleveland[~cleveland.isin(['?'])]
data.loc[280:]

In [ ]:
# drop rows with NaN values from DataFrame
data = data.dropna(axis=0)
data.loc[280:]

In [ ]:
# print the shape and data type of the dataframe
print (data.shape)
print (data.dtypes)

In [ ]:
# transform data to numeric to enable further analysis
data = data.apply(pd.to_numeric)
data.dtypes

In [ ]:
# print data characteristics, usings pandas built-in describe() function
data.describe()

In [ ]:
# plot histograms for each variable
data.hist(figsize = (12, 12))
plt.show()

In [12]:
# create X and Y datasets for training
from sklearn import model_selection
X = np.array(data.drop(['class'], axis=1))  # axis=1 indicates you're dropping a column

# Get the target variable 'class' as y
y = np.array(data['class'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# convert the data to categorical labels

from tensorflow.keras.utils import to_categorical

# Convert the labels to categorical (one-hot encoding)
Y_train = to_categorical(y_train, num_classes=None)
Y_test = to_categorical(y_test, num_classes=None)
print (Y_train.shape)
print (Y_train[:10])

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Ensure the necessary functions are defined
def create_model():
    model = Sequential()  # Create a sequential model
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Input layer and first hidden layer
    model.add(Dense(64, activation='relu'))  # Second hidden layer
    model.add(Dense(5, activation='softmax'))  # Output layer for 5 classes
    
    # Initialize the Adam optimizer with a learning rate of 0.001
    adam = Adam(learning_rate=0.001)
    
    # Compile the model with categorical crossentropy loss and accuracy metrics
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return model

# Create and print the model summary
model = create_model()
print(model.summary())


In [ ]:
# fit the model to the training data
model.fit(X_train, Y_train, epochs=100, batch_size=10, verbose = 1)

In [ ]:
# convert into binary classification problem - heart disease or no heart disease
Y_train_binary = y_train.copy()
Y_test_binary = y_test.copy()

Y_train_binary[Y_train_binary > 0] = 1
Y_test_binary[Y_test_binary > 0] = 1

print (Y_train_binary[:20])

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def create_binary_model():
    model = Sequential()
    model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Example input layer
    model.add(Dense(64, activation='relu'))  # Hidden layer
    model.add(Dense(1, activation='sigmoid'))  # Output layer (binary classification)
    
    # Use learning_rate instead of lr
    adam = Adam(learning_rate=0.001)
    
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return model

binary_model = create_binary_model()
print(binary_model.summary())


In [ ]:
# fit the binary model on the training data
binary_model.fit(X_train, Y_train_binary, epochs=100, batch_size=10, verbose = 1)

In [ ]:
# generate classification report using predictions for categorical model
from sklearn.metrics import classification_report, accuracy_score

categorical_pred = np.argmax(model.predict(X_test), axis=1)

print('Results for Categorical Model')
print(accuracy_score(y_test, categorical_pred))
print(classification_report(y_test, categorical_pred))

In [ ]:
# generate classification report using predictions for binary model 
binary_pred = np.round(binary_model.predict(X_test)).astype(int)

print('Results for Binary Model')
print(accuracy_score(Y_test_binary, binary_pred))
print(classification_report(Y_test_binary, binary_pred))